#### CV_mp 虛擬環境
* model

In [ ]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2

base_options = python.BaseOptions(
    model_asset_path='models/blaze_face_short_range.tflite')
options = vision.FaceDetectorOptions(base_options=base_options)
detector = vision.FaceDetector.create_from_options(options)
img = mp.Image.create_from_file("images/face.jpg")
detection_result = detector.detect(img)
if detection_result:
    img = img.numpy_view()
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    height, width, _ = img.shape
    for detection in detection_result.detections:
        bbox = detection.bounding_box
        x1 = bbox.origin_x
        y1 = bbox.origin_y
        x2 = bbox.origin_x + bbox.width
        y2 = bbox.origin_y + bbox.height
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 255), 2)
        for keypoint in detection.keypoints:
            cx = int(keypoint.x * width)
            cy = int(keypoint.y * height)
            cv2.circle(img, (cx, cy), 2, (255, 0, 255), 2)

        category = detection.categories[0]
        probability = round(category.score, 2)
        score = str(probability * 100) + "%"
        cv2.putText(img, score, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

cv2.imshow("MediaPipe Face Detection", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
from mediapipe import solutions  # 匯入 Mediapipe 的 solutions 模組
from mediapipe.framework.formats import landmark_pb2  # 處理關鍵點座標格式
import numpy as np  # 用於數值運算
import mediapipe as mp  # MediaPipe 核心函式庫
from mediapipe.tasks import python  # MediaPipe 任務模組
from mediapipe.tasks.python import vision  # MediaPipe 視覺模組
import cv2  # OpenCV 用於影像處理

# 建立 MediaPipe 臉部偵測器 (FaceLandmarker)
base_options = python.BaseOptions(
    model_asset_path='models/face_landmarker.task')  # 設定模型路徑
options = vision.FaceLandmarkerOptions(
    base_options=base_options,
    output_face_blendshapes=True,  # 啟用 Blendshapes 偵測
    output_facial_transformation_matrixes=True,  # 啟用 3D 變形矩陣輸出
    num_faces=1  # 限制偵測 1 張臉
)
detector = vision.FaceLandmarker.create_from_options(options)  # 初始化偵測器
# 讀取圖片並進行臉部偵測
image = mp.Image.create_from_file("images/face.jpg")  # 載入影像
detection_result = detector.detect(image)  # 執行偵測
# 取得臉部特徵點資料
face_landmarks_list = detection_result.face_landmarks  # 取得臉部特徵點資料
# 判斷是否有偵測到臉部
if face_landmarks_list:
    annotated_image = np.copy(image.numpy_view())  # 複製影像，避免修改原圖
    # 遍歷所有偵測到的臉部 (支援多張臉)
    for idx in range(len(face_landmarks_list)):
        face_landmarks = face_landmarks_list[idx]  # 取得第 idx 張臉的特徵點
        # 建立 MediaPipe 標準的 Landmark 格式
        face_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
        face_landmarks_proto.landmark.extend([
            landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in face_landmarks
        ])
        # 在影像上繪製臉部網格 (Tessellation)
        solutions.drawing_utils.draw_landmarks(
            image=annotated_image,
            landmark_list=face_landmarks_proto,
            connections=mp.solutions.face_mesh.FACEMESH_TESSELATION,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp.solutions.drawing_styles.get_default_face_mesh_tesselation_style()
        )
        # 在影像上繪製臉部輪廓 (Contours)
        solutions.drawing_utils.draw_landmarks(
            image=annotated_image,
            landmark_list=face_landmarks_proto,
            connections=mp.solutions.face_mesh.FACEMESH_CONTOURS,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp.solutions.drawing_styles.get_default_face_mesh_contours_style()
        )
        # 在影像上繪製虹膜 (Irises)
        solutions.drawing_utils.draw_landmarks(
            image=annotated_image,
            landmark_list=face_landmarks_proto,
            connections=mp.solutions.face_mesh.FACEMESH_IRISES,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp.solutions.drawing_styles.get_default_face_mesh_iris_connections_style()
        )

# 使用 OpenCV 顯示標記後的影像
cv2.imshow("MediaPipe FaceMesh", cv2.cvtColor(
    annotated_image, cv2.COLOR_RGB2BGR))
cv2.waitKey(0)  # 等待按鍵以關閉視窗
cv2.destroyAllWindows()  # 關閉所有 OpenCV 視窗


In [ ]:
from mediapipe import solutions  # 匯入 Mediapipe 的 solutions 模組
from mediapipe.framework.formats import landmark_pb2  # 處理關鍵點座標格式
import numpy as np  # 用於數值運算
import mediapipe as mp  # MediaPipe 核心函式庫
from mediapipe.tasks import python  # MediaPipe 任務模組
from mediapipe.tasks.python import vision  # MediaPipe 視覺模組
import cv2  # OpenCV 用於影像處理

# 設定模型路徑和偵測參數，允許最多 2 隻手
base_options = python.BaseOptions(
    model_asset_path='models/hand_landmarker.task')  # 設定模型路徑
options = vision.HandLandmarkerOptions(
    base_options=base_options, num_hands=2)  # 設定偵測參數，最多允許2隻手
detector = vision.HandLandmarker.create_from_options(options)  # 建立手部標記偵測器
# 載入輸入影像
image = mp.Image.create_from_file(
    "images/hands.jpg")  # 從檔案讀取影像並轉換為 Mediapipe 影像格式
# 偵測影像中的手部標記點
detection_result = detector.detect(image)  # 使用偵測器對影像進行手部標記偵測
# 處理偵測結果
if len(detection_result.hand_landmarks) > 0:  # 如果偵測到手部標記點
    annotated_image = np.copy(image.numpy_view())  # 建立影像副本
    for hand_landmarks in detection_result.hand_landmarks:  # 遍歷所有偵測到的手部標記點
        hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()  # 建立 Mediapipe 格式的標記點資料結構
        hand_landmarks_proto.landmark.extend([
            landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
        ])  # 填充標記點資料
        solutions.drawing_utils.draw_landmarks(
            annotated_image,  # 要繪製的影像
            hand_landmarks_proto,  # 手部標記點資料
            solutions.hands.HAND_CONNECTIONS,  # 預設的手部標記點連線規則
            solutions.drawing_styles.get_default_hand_landmarks_style()  # 預設的標記點樣式
        )  # 使用 Mediapipe 的繪圖工具繪製手部標記點與連接線

cv2.imshow("MediaPipe Hands", cv2.cvtColor(annotated_image,
           cv2.COLOR_RGB2BGR))  # 使用 OpenCV 顯示影像，轉換為 BGR 色彩模式
cv2.waitKey(0)  # 等待使用者按下任意鍵後關閉視窗
cv2.destroyAllWindows()  # 關閉所有 OpenCV 視窗


In [1]:
import mediapipe as mp  # 匯入 Mediapipe 核心函式庫
from mediapipe import solutions  # 匯入 Mediapipe 的 solutions 模組
from mediapipe.framework.formats import landmark_pb2  # 匯入 Mediapipe 的關鍵點格式
from mediapipe.tasks import python  # 匯入 Mediapipe 的 Python 任務 API
from mediapipe.tasks.python import vision  # 匯入 Mediapipe 的視覺處理模組
import numpy as np  # 匯入 NumPy 進行影像處理
import cv2  # 匯入 OpenCV 進行影像顯示

# 初始化 Mediapipe 姿勢偵測器
base_options = python.BaseOptions(
    model_asset_path='models/pose_landmarker_full.task')  # 設定模型路徑
options = vision.PoseLandmarkerOptions(
    base_options=base_options, output_segmentation_masks=True)  # 設定偵測參數
detector = vision.PoseLandmarker.create_from_options(options)  # 建立姿勢偵測器
# 讀取輸入影像
mp_image = mp.Image.create_from_file(
    "images/pose.jpg")  # 從檔案讀取影像並轉換為 Mediapipe 影像格式
# 進行姿勢偵測
detection_result = detector.detect(mp_image)  # 使用偵測器對影像進行分析
# 處理偵測結果
if len(detection_result.pose_landmarks) > 0:  # 如果偵測到姿勢關鍵點
    print("偵測到的姿勢數量:", len(detection_result.pose_landmarks))  # 印出偵測到的姿勢數量
    annotated_image = np.copy(mp_image.numpy_view())  # 建立影像副本
    for pose_landmarks in detection_result.pose_landmarks:  # 遍歷所有偵測到的姿勢關鍵點
        pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()  # 建立 Mediapipe 格式的關鍵點資料結構
        pose_landmarks_proto.landmark.extend([
            landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
        ])  # 填充關鍵點資料
        solutions.drawing_utils.draw_landmarks(
            annotated_image,  # 要繪製的影像
            pose_landmarks_proto,  # 姿勢關鍵點資料
            solutions.pose.POSE_CONNECTIONS,  # 預設的姿勢關鍵點連線規則
            solutions.drawing_styles.get_default_pose_landmarks_style()  # 預設的關鍵點樣式
        )  # 使用 Mediapipe 的繪圖工具繪製姿勢關鍵點與連接線

# 使用 OpenCV 顯示影像，轉換為 BGR 色彩模式
cv2.imshow('Image', cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))
cv2.waitKey(0)  # 等待使用者按下任意鍵後關閉視窗
cv2.destroyAllWindows()  # 關閉所有 OpenCV 視窗


c:\Users\q5j6j\miniconda3\envs\CV_mp\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


偵測到的姿勢數量: 1


In [1]:
try:
    from tflite_runtime.interpreter import Interpreter
except ImportError:
    from tensorflow.lite.python.interpreter import Interpreter
import cv2
import numpy as np

model_path = "models/mobilenet_v1_1.0_224_quantized_1_metadata_1.tflite"
label_path = "models/labels.txt"
label_names = []
with open(label_path, "r") as f:
    for line in f.readlines():
        label_names.append(line.strip())
interpreter = Interpreter(model_path)
print("成功載入模型...")
interpreter.allocate_tensors()
_, height, width, _ = interpreter.get_input_details()[0]["shape"]
print("影像尺寸: (", width, ",", height, ")")
image = cv2.imread("images/dog.jpg")
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image_resized = cv2.resize(image_rgb, (width, height))
input_data = np.expand_dims(image_resized, axis=0)
interpreter.set_tensor(
    interpreter.get_input_details()[0]["index"], input_data)
interpreter.invoke()
output_details = interpreter.get_output_details()[0]
output = np.squeeze(interpreter.get_tensor(output_details["index"]))
label_id = np.argmax(output)
scale, zero_point = output_details["quantization"]
prob = scale * (output[label_id] - zero_point)
classification_label = label_names[label_id]
print("分類名稱 =", classification_label)
print("影像可能性 =", np.round(prob*100, 2), "%")



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\q5j6j\miniconda3\envs\CV_mp\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\q5j6j\miniconda3\envs\CV_mp\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\q5j6j\miniconda3\envs\CV_mp\Lib\site-packages\ipykernel\kernelapp.py", line 758, in start
    self.io_loop.start()
  File "c:

AttributeError: _ARRAY_API not found

SystemError: <built-in method CreateWrapperFromFile of PyCapsule object at 0x000001FDD373CDE0> returned a result with an exception set